# 维数灾难 curse of dimentionality
部分机器学习问题涉及数千、数百万维特征，不仅让训练过程变得非常缓慢，有时还很难找到一个很好的解。
训练缓慢无须赘言，为什么还很难寻解呢？主要是过拟合风险大，原因如下：
- 高维超正方体中，大多数点都分布在边界处。
- 高维数据集有很大风险分布的非常稀疏。

针对维数爆炸解决方案：
- **增大训练集：**训练实例的数量随着维度的数量呈指数增长。
- **降维：**必然丢失信息,工作流水线更复杂、更难维护。

虽然降维有时会过滤噪声数据，但建议先尝试原始数据训练，因为降维使训练速度加快，但并不总是得出更好的训练效果，如速度太慢再考虑降维。此外，在降维还有利于数据可视化。
# 投影 projection
训练实例并不是所有维度上均匀分布的，许多特征几乎是常数，而其他特征则高度相关，所以训练实例几近分布于高维空间的某低维子空间内。
## 主成分分析（PCA）
### 主要原理
1. 寻找接近数据分布的超平面（保留最大方差，投影后信息损失最少）。
2. 将数据都投影到步骤1中的超平面上。
3. 主成分数量的选取：方差解释率（Explained Variance Ratio）累加和达到设定阈值的维度数量作为保留的主成分数量。其中，方差解释率表示位于每个主成分轴上的数据集方差的比例。

### 注意事项
- 数据集需中心化处理，PCA 假定数据集以原点为中心。
- 主成分方向并不稳定，但平面通常保持不变。
- PCA投影的逆变换与原始数据存在偏差，所以选取最优偏差也是PCA优化的一个方向。

### 示例
PCA：

In [20]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

X,y = load_boston(return_X_y=True)
pca = PCA() # 自带中心化处理
pca.fit(X)
n_pc = np.argmax(np.cumsum(pca.explained_variance_ratio_) > 0.95) + 1
pca = PCA(n_components=n_pc)
x_pc = pca.fit_transform(X)
X2 = pca.inverse_transform(x_pc)
mean_squared_error(X,X2)

91.59711211033573

增量PCA：

In [12]:
from sklearn.decomposition import IncrementalPCA
n_batches = 3
inc_pca=IncrementalPCA(n_components=2)
for X_batch in np.array_split(X,n_batches):
    inc_pca.partial_fit(X_batch)
inc_pca.transform(X)

array([[-119.80985122,    5.54437069],
       [-168.83810838,  -10.58038816],
       [-169.28890585,  -14.31807157],
       ...,
       [-138.31886412,   -1.51625547],
       [-137.44054453,   -4.80866635],
       [-139.13708385,   -1.46521395]])

```
# 硬盘读取
X = np.memmap(filename,
          dtype='float32',
          mode='readonly',
          shape=(m,n))
batch_size = m//n_batches
inc_pca = IncrementalPCA(n_components=100,batch_size=batch_size)
inc_pca.fit(X)
```

随机PCA：

In [14]:
pca_rnd = PCA(n_components=2,svd_solver='randomized')

核PCA：

In [24]:
from sklearn.decomposition import KernelPCA
rbf_pca = KernelPCA(n_components=2,kernel='rbf',gamma=0.04,fit_inverse_transform=False)
rbf_pca.fit_transform(X)

array([[-0.01443817, -0.01333743],
       [-0.01443829, -0.01333755],
       [-0.01494978, -0.01390031],
       ...,
       [-0.01702695, -0.01617612],
       [-0.01727138, -0.01643593],
       [-0.01682136, -0.01593888]])

# 流形学习 Manifold Learning
投影可能存在的缺陷是经过投影后部分不同类别数据点交错重叠影响建模效果，另一种方法就是流形学习，其依赖于流形猜想（manifold assumption），也被称为流形假设（manifold hypothesis）,能利用流行学习建模的假设如下：
- 多数高维数据集大都靠近一个更低维的流形。
- 当前建模任务在流形的较低维空间中表示会变得更简单。
## 局部线性嵌入 LLE
LLE 首先测量每个训练实例与其最近邻（c.n.）之间的线性关系，然后寻找能最好地保留这些局部关系的训练集的低维表示（稍后会详细介绍） 。这使得它特别擅长展开扭曲的流形，尤其是在没有太多噪音的情况下。

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding
lle=LocallyLinearEmbedding(n_components=2,n_neighbors=10)
X_reduced=lle.fit_transform(X)

还有很多其他的降维方法，Scikit-Learn 支持其中的好几种。这里是其中最流行的：

多维缩放（MDS）在尝试保持实例之间距离的同时降低了维度（参见图 8-13）
Isomap 通过将每个实例连接到最近的邻居来创建图形，然后在尝试保持实例之间的测地距离时降低维度。
t-分布随机邻域嵌入（t-Distributed Stochastic Neighbor Embedding，t-SNE）可以用于降低维​​度，同时试图保持相似的实例临近并将不相似的实例分开。它主要用于可视化，尤其是用于可视化高维空间中的实例（例如，可以将MNIST图像降维到 2D 可视化）。
线性判别分析（Linear Discriminant Analysis，LDA）实际上是一种分类算法，但在训练过程中，它会学习类之间最有区别的轴，然后使用这些轴来定义用于投影数据的超平面。LDA 的好处是投影会尽可能地保持各个类之间距离，所以在运行另一种分类算法（如 SVM 分类器）之前，LDA 是很好的降维技术。